# Proyecto: Análisis del mercado inmobiliario de la Ciudad de Buenos Aires

Mentor: Javier Lezama.

javitolez@gmail.com

En este proyecto vamos a trabajar con un dataset de propiedades en venta publicadas en el portal www.Properati.com.ar.

# El dataset: propiedades en venta en Ciudad de Buenos Aires.

En este proyecto, trabajaremos con una muestra del conjunto de datos de propiedades en venta mencionado. En este dataset, cada fila es una propiedad en venta. 

#Los atributos

A continuación vamos a describir los atributos que consideramos en esta muestra:
* id: id de la propiedad
* created_on: fecha en la que la propiedad ingresó al sitio
* operation: alquiler (rent) o venta (sell)
* property_type: tipo de propiedad (casa, departamento, ph, etcétera)
* place_with_parent_names: nombre del lugar donde se encuentra la propiedad según el publicador
* lat-lon: coordenadas concatenadas
* lat: latitud
* lon: longitud
* price: precio en la moneda especificada en currency
* currency: divisa en la que está expresada la publicación
* price_aprox_usd: precio aproximado en dólares estadounidenses
* surface_total_in_m2: superficie total (en metros cuadrados)
* surface_covered_in_m2: superficie cubierta (en metros cuadrados)
* price_usd_per_m2: precio por metro cuadrado en dólares (precio dólares / superficie)
* floor: número de piso (si corresponde)
* rooms: cantidad de ambientes
* expenses: expensas (si corresponde)
* barrio: barrio según cartografía oficial
* properati_url: url de la publicación en Properati

Objetivos del practico introducción al aprendizaje automático:

El objetivo de este practico es trabajar con el mismo dateset trabajado en el práctico dos, específicamente el objetivo es que se introduzcan en el desarrollo de un esquema para hacer tareas de aprendizaje automático: selección de un modelo, ajuste de hiperparámetros y evaluación.

Para eso trabajaremos con el dataset resultante del practico dos para armar nuestros primeros modelos para predecir el precio de las propiedades en dólares.

# Manos a la obra

Partiremos con el dataset resultante del práctico dos. El práctico está basado en el práctico final de la materia introducción al aprendizaje automatico del año 2018.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from matplotlib.colors import ListedColormap
from sklearn.datasets import load_boston, load_breast_cancer, load_iris
from sklearn import linear_model , metrics
from sklearn.linear_model import LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix, mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline

np.random.seed(1234)  # Setup seed to be more deterministic

%matplotlib inline

In [2]:
df_prop = pd.read_csv('df_bin.csv')
df_prop.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7716 entries, 0 to 7715
Data columns (total 79 columns):
Unnamed: 0                                                                       7716 non-null int64
created_on                                                                       7716 non-null object
operation                                                                        7716 non-null object
property_type                                                                    7716 non-null int64
place_with_parent_names                                                          7716 non-null object
lat-lon                                                                          7716 non-null object
lat                                                                              7716 non-null float64
lon                                                                              7716 non-null float64
price_aprox_usd                                                              

# Ejercicio 1: División de datos en conjuntos de entrenamiento y evaluación.

La primer tarea consiste en dividir el conjunto de datos cargados en el apartado anterior en conjuntos de entrenamiento (o training) y evaluación (o test).

Se define un dataframe con los features que se consideran importantes para la definición del precio de la propiedad

In [3]:
df_prop_gt = df_prop[['surface_total_in_m2','barrio_cod','property_type','rooms','surface_covered_in_m2','price_aprox_usd']]
df_prop_gt.head()

,surface_total_in_m2,barrio_cod,property_type,rooms,surface_covered_in_m2,price_aprox_usd
0,130.0,20,1,4.000000,120.0,340000.00
1,70.0,20,1,2.422019,70.0,180000.00
2,532.0,20,2,6.000000,334.0,1200000.00
3,72.0,20,0,3.000000,72.0,219000.00
4,40.0,20,1,1.000000,36.0,119124.77


Se separa el set de datos tomando el 80% para entrenamiento y el 20% para validación

In [12]:
#Utilizamos aproximadamente 90% de los datos para entrenamiento y 20% para validación
#entradas = len(df_prop)
#data_div = round(entradas*0.9)

#shuff_data = np.random.permutation(506)
#shuff_train = shuff_data[:data_div]
#shuff_val = shuff_data[data_div:]

X = df_prop_gt.drop('price_aprox_usd', axis=1)
y = df_prop_gt.price_aprox_usd

from sklearn.model_selection import train_test_split
#Random state sirve para sembrar una "Seed" de como se distribuyen los datos para el algoritmo
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)

# Necesario para poder hacer un regresor por feature
#features_names = df_prop.columns
#feature_map = {feature: idx for idx, feature in enumerate(features_names)}


# Ejercicio 2: Elección de un modelo de regresión.

https://gist.github.com/lprowell/81ab813d84096290c5a833a8c7731355

Basándose en lo visto en el teórico escojan y justifiquen un modelo de aprendizaje automático de regresión, para predecir el precio de las propiedades en dólares. Selección de regulizador, selección de la función de coste, etc.

- Elección Modelo Regresión Lineal

In [5]:
# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
#model = LinearRegression()
model = linear_model.Ridge(alpha=0.1)
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_valid)))

Media del error cuadrado para entrenamiento: 1381303235.87
Media del error cuadrado para validación: 1190908525.00


In [6]:
# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
#model = LinearRegression()
model = linear_model.Lasso(alpha=0.1)
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_valid)))

Media del error cuadrado para entrenamiento: 1381303236.00
Media del error cuadrado para validación: 1190908496.52


In [38]:
# Entrenamos un clasificador utilizando sólo ese atributo sobre el conjunto de entrenamiento (X_train, y_train)
#model = LinearRegression()
model = linear_model.LinearRegression()
model.fit(X_train, y_train)

# Evaluamos el desempeño del clasificador utilizando la media del error cuadrado (MSE o mean squared error)
# sobre el conjunto de datos de entrenamiento (X_train, y_train) y lo comparamos con el de validación (X_val, y_val)
# Mientras más cercano a cero mejor
print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_valid)))

Media del error cuadrado para entrenamiento: 1381303235.86
Media del error cuadrado para validación: 1190908612.83


- Elección Modelo Regresión Polinomial

In [7]:
polynomial_degree = 2 # TODO: Grado del polinomio.
alpha = 0.1 #TODO: Parámetro de regularización. También denominado como parámetro `lambda`.

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(X_train)
X_poly_train = poly_features.transform(X_train)
X_poly_val = poly_features.transform(X_valid)

model = linear_model.Ridge(alpha=alpha)
model.fit(X_poly_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_poly_val)))

Media del error cuadrado para entrenamiento: 1242441149.22
Media del error cuadrado para validación: 1171208356.38


C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\linear_model\ridge.py:125: LinAlgWarning: scipy.linalg.solve
Ill-conditioned matrix detected. Result is not guaranteed to be accurate.
Reciprocal condition number2.489736e-17
  overwrite_a=True).T


In [8]:
polynomial_degree = 2 # TODO: Grado del polinomio.
alpha = 0.1 #TODO: Parámetro de regularización. También denominado como parámetro `lambda`.

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(X_train)
X_poly_train = poly_features.transform(X_train)
X_poly_val = poly_features.transform(X_valid)

model = linear_model.Lasso(alpha=alpha)
model.fit(X_poly_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_poly_val)))

Media del error cuadrado para entrenamiento: 1244541724.98
Media del error cuadrado para validación: 1185016668.36


C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\linear_model\coordinate_descent.py:492: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


In [9]:
polynomial_degree = 2 # TODO: Grado del polinomio.

poly_features = PolynomialFeatures(polynomial_degree)
poly_features.fit(X_train)
X_poly_train = poly_features.transform(X_train)
X_poly_val = poly_features.transform(X_valid)

model = linear_model.LinearRegression()
model.fit(X_poly_train, y_train)

print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_poly_val)))

Media del error cuadrado para entrenamiento: 1242441108.04
Media del error cuadrado para validación: 1171223092.99


# Ejercicio 3: Selección de hiperparámetros.

Utilizando búsqueda exhaustiva (grid search) con 5-fold cross-validation y utilizando como métrica el área bajo la curva de ROC (o ROC-AUC), hagan una selección de los mejores hiperparámetros para su conjunto de datos y el modelo que hayan elegido en el apartado anterior.

In [13]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### GridSearchCV Regresión Lineal

In [20]:
params={'alpha': [500,100,50,25,10,4,2,1.0,0.8,0.5,0.3,0.2,0.1,0.05,0.02,0.01]}
estimators = linear_model.Ridge()

model = GridSearchCV(estimators,params,cv=5,verbose = 1, scoring = 'neg_mean_squared_error')         
model.fit(X_train, y_train)

#Acá arme un dataframe con los resultados, pero la clave sería ver el reporte que armó el profe en clase 
#pd.DataFrame(model.cv_results_)

#print("# Exploración de hiperparámetros para función de coste \"%s\"" % loss, end="\n\n")

#https://machinelearningmastery.com/how-to-tune-algorithm-parameters-with-scikit-learn/
#print("Mejor conjunto de parámetros:")
print(model.best_score_)
print(model.best_estimator_.alpha)


Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


-1396847524.5321035
500


[Parallel(n_jobs=1)]: Done  80 out of  80 | elapsed:    0.3s finished


### GridSearchCV Regresión Polinomial

In [27]:
from matplotlib import pyplot as plt
import numpy as np
from scipy import stats
from sklearn.base import BaseEstimator


In [28]:
for polynomial_degree in (1,2,3,4,5,6,7,8,9,10,11) : # TODO: Probar distintos grados del polinomio

    poly_features = PolynomialFeatures(polynomial_degree)
    poly_features.fit(X_train)
    X_poly_train = poly_features.transform(X_train)
    X_poly_val = poly_features.transform(X_valid)

    model = LinearRegression()
    model.fit(X_poly_train, y_train)

    print('Media del error cuadrado para entrenamiento: %.2f' % 
      mean_squared_error(y_train, model.predict(X_poly_train)))
    print('Media del error cuadrado para validación: %.2f' %
      mean_squared_error(y_valid, model.predict(X_poly_val)))

Media del error cuadrado para entrenamiento: 1381303235.86
Media del error cuadrado para validación: 1190908612.83
Media del error cuadrado para entrenamiento: 1242441108.04
Media del error cuadrado para validación: 1171223092.99
Media del error cuadrado para entrenamiento: 1060527371.85
Media del error cuadrado para validación: 1430872670.30
Media del error cuadrado para entrenamiento: 949188925.35
Media del error cuadrado para validación: 5105420223.39
Media del error cuadrado para entrenamiento: 1451565477.46
Media del error cuadrado para validación: 138917023936713.67
Media del error cuadrado para entrenamiento: 31111976874.91
Media del error cuadrado para validación: 19445279789722632.00
Media del error cuadrado para entrenamiento: 32042949762.42
Media del error cuadrado para validación: 79625002589411824.00
Media del error cuadrado para entrenamiento: 78316313720.32
Media del error cuadrado para validación: 33542228253884724.00
Media del error cuadrado para entrenamiento: 1219757

In [33]:
def PolynomialRegressionLasso(degree=2, **kwargs):
    return make_pipeline(PolynomialFeatures(degree), LinearRegression(**kwargs))

In [34]:
param_grid = {'polynomialfeatures__degree': np.arange(9)}

poly_grid = GridSearchCV(PolynomialRegression(), param_grid, 
                         cv=5, 
                         scoring='neg_mean_squared_error', 
                         verbose=3,
                         n_jobs =2
                        ) 

poly_grid.fit(X_train, y_train)


NameError: name 'PolynomialRegression' is not defined

In [31]:
print("# Exploración de hiperparámetros para función de coste mean_square\"%s\"\n\n")
    
print("Mejor conjunto de parámetros:")
print(poly_grid.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = poly_grid.cv_results_['mean_test_score']
stds = poly_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, poly_grid.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

# Exploración de hiperparámetros para función de coste mean_square"%s"


Mejor conjunto de parámetros:


NameError: name 'poly_grid' is not defined

In [35]:
from sklearn.linear_model import Ridge

In [36]:
def PolynomialRegressionRidge(degree=2, alpha=1,random_state=42):
    return make_pipeline(PolynomialFeatures(degree), Ridge(random_state,alpha))

In [39]:
param_grid = {'polynomialfeatures__degree': np.arange(9),
              'ridge__alpha': [0.1, 0.01, 0.001,1]
             }

poly_grid = GridSearchCV(PolynomialRegressionRidge(), param_grid, 
                         cv=5, 
                         scoring='neg_mean_squared_error', 
                         verbose=3,
                         n_jobs =2
                        ) 

poly_grid.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done 180 out of 180 | elapsed:  1.8min finished
C:\Users\Gabriel H\.conda\envs\diplodatos\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('ridge', Ridge(alpha=42, copy_X=True, fit_intercept=1, max_iter=None, normalize=False,
   random_state=None, solver='auto', tol=0.001))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6, 7, 8]), 'ridge__alpha': [0.1, 0.01, 0.001, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [68]:
print("# Exploración de hiperparámetros para función de coste mean_square\"%s\"\n\n")
    
print("Mejor conjunto de parámetros:")
print(poly_grid.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = poly_grid.cv_results_['mean_test_score']
stds = poly_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, poly_grid.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

# Exploración de hiperparámetros para función de coste mean_square"%s"


Mejor conjunto de parámetros:
{'polynomialfeatures__degree': 1, 'ridge__alpha': 1}

Puntajes de la grilla:

Exactitud: -34755673853.317 (+/-3123301732792002560.000) para los parámetros {'polynomialfeatures__degree': 0, 'ridge__alpha': 0.1}
Exactitud: -34755673853.317 (+/-3123301732792002560.000) para los parámetros {'polynomialfeatures__degree': 0, 'ridge__alpha': 0.01}
Exactitud: -34755673853.317 (+/-3123301732792002560.000) para los parámetros {'polynomialfeatures__degree': 0, 'ridge__alpha': 0.001}
Exactitud: -34755673853.317 (+/-3123301732792002560.000) para los parámetros {'polynomialfeatures__degree': 0, 'ridge__alpha': 1}
Exactitud: -1398589603.164 (+/-8129831462108328.000) para los parámetros {'polynomialfeatures__degree': 1, 'ridge__alpha': 0.1}
Exactitud: -1398590012.836 (+/-8129908262872359.000) para los parámetros {'polynomialfeatures__degree': 1, 'ridge__alpha': 0.01}
Exactitud: -1398590053.807 (+/-81

In [72]:
from sklearn.linear_model import Lasso

In [73]:
def PolynomialRegressionLasso(degree=2, alpha=1,random_state=42):
    return make_pipeline(PolynomialFeatures(degree), Lasso(random_state,alpha))

In [74]:
param_grid = {'polynomialfeatures__degree': np.arange(9),
              'lasso__alpha': [0.1, 0.01, 0.001,1]
             }

poly_grid = GridSearchCV(PolynomialRegressionLasso(), param_grid, 
                         cv=5, 
                         scoring='neg_mean_squared_error', 
                         verbose=3,
                         n_jobs =2
                        ) 

poly_grid.fit(X_train, y_train)

Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=2)]: Using backend LokyBackend with 2 concurrent workers.
/home/manuel/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
/home/manuel/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=2)]: Done  39 tasks      | elapsed:  1.1min
[Parallel(n_jobs=2)]: Done 135 tasks      | elapsed:  5.0min
[Parallel(n_jobs=2)]: Done 180 out of 180 | elapsed:  6.7min finished
/home/manuel/anaconda3/envs/diplodatos/lib/python3.7/site-packages/sklearn/model_selectio

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('polynomialfeatures', PolynomialFeatures(degree=2, include_bias=True, interaction_only=False)), ('lasso', Lasso(alpha=42, copy_X=True, fit_intercept=1, max_iter=1000, normalize=False,
   positive=False, precompute=False, random_state=None, selection='cyclic',
   tol=0.0001, warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=2,
       param_grid={'polynomialfeatures__degree': array([0, 1, 2, 3, 4, 5, 6, 7, 8]), 'lasso__alpha': [0.1, 0.01, 0.001, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='neg_mean_squared_error', verbose=3)

In [75]:
print("# Exploración de hiperparámetros para función de coste mean_square\"%s\"\n\n")
    
print("Mejor conjunto de parámetros:")
print(poly_grid.best_params_, end="\n\n")

print("Puntajes de la grilla:", end="\n\n")
means = poly_grid.cv_results_['mean_test_score']
stds = poly_grid.cv_results_['std_test_score']
for mean, std, params in zip(means, stds, poly_grid.cv_results_['params']):
    print("Exactitud: %0.3f (+/-%0.03f) para los parámetros %r" % (mean, std ** 2, params))
print()

# Exploración de hiperparámetros para función de coste mean_square"%s"


Mejor conjunto de parámetros:
{'lasso__alpha': 1, 'polynomialfeatures__degree': 1}

Puntajes de la grilla:

Exactitud: -34755673853.317 (+/-3123301732792002560.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 0}
Exactitud: -1398588524.460 (+/-8129787163895926.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 1}
Exactitud: -1603654800.587 (+/-114380738380813248.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 2}
Exactitud: -1996338199.294 (+/-698137667876569472.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 3}
Exactitud: -2204793367.064 (+/-887717679641045376.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 4}
Exactitud: -9600076482.782 (+/-242961871987110445056.000) para los parámetros {'lasso__alpha': 0.1, 'polynomialfeatures__degree': 5}
Exactitud: -16045179189.593 (+/-8216

In [ ]:
estimator = PolynomialRegression()
degrees = np.arange(1, 10)
cv_model = GridSearchCV(estimator,
                        param_grid={'deg': degrees},
                        scoring='mean_squared_error')
cv_model.fit(X_train, y_train)

# Ejercicio 4: Métricas sobre el conjunto de evaluación

Una vez encontrados los mejores hiperparámetros para el modelo seleccionado en los apartados anteriores se evalúa el modelo final entrenado sobre el conjunto de datos de evaluación seleccionado en el ejercicio 1. Pueden utilizar las métricas que crean convenientes.

# Informe final

Presentar un informe final con los datos inferidos del dataset utilizando herramientas gráficas y escritas teniendo en cuenta los destinatarios. En este caso colegas de la diplomatura para preparar el informe con un background adecuado al destinatario.